In [1]:
import glob
import sys
import random
import numpy as np
from tensorflow.keras import * 

In [2]:
ls -l /data/cassandra/

total 732
drwxrwxr-x  2 abasar abasar   4096 Sep 29 08:44 bin/
-rw-rw-r--  1 abasar abasar    262 Sep 29 08:44 build.properties.default
-rw-rw-r--  1 abasar abasar 107961 Sep 29 08:44 build.xml
-rw-rw-r--  1 abasar abasar   4832 Sep 29 08:44 CASSANDRA-14092.txt
-rw-rw-r--  1 abasar abasar 395833 Sep 29 08:44 CHANGES.txt
drwxrwxr-x  3 abasar abasar   4096 Sep 29 08:44 conf/
-rw-rw-r--  1 abasar abasar   1169 Sep 29 08:44 CONTRIBUTING.md
drwxrwxr-x  3 abasar abasar   4096 Sep 29 08:44 debian/
drwxrwxr-x  4 abasar abasar   4096 Sep 29 08:44 doc/
-rw-rw-r--  1 abasar abasar   5895 Sep 29 08:44 eclipse_compiler.properties
drwxrwxr-x  3 abasar abasar   4096 Sep 29 08:44 examples/
drwxrwxr-x  4 abasar abasar   4096 Sep 29 08:44 ide/
drwxrwxr-x  4 abasar abasar   4096 Sep 29 08:44 lib/
-rw-rw-r--  1 abasar abasar  11609 Sep 29 08:44 LICENSE.txt
-rw-rw-r--  1 abasar abasar 124736 Sep 29 08:44 NEWS.txt
-rw-rw-r--  1 abasar abasar   2600 Sep 29 08:44 NOTICE.txt
drwxrwxr-x  3 abasar abasar   4096 

In [3]:
from pathlib import Path

filenames = []
for filename in Path("/data/cassandra/src/").glob('**/*.java'):
    print(filename)
    filenames.append(filename)

/data/cassandra/src/java/org/apache/cassandra/service/LoadBroadcaster.java
/data/cassandra/src/java/org/apache/cassandra/service/GCInspectorMXBean.java
/data/cassandra/src/java/org/apache/cassandra/service/EchoVerbHandler.java
/data/cassandra/src/java/org/apache/cassandra/service/GCInspector.java
/data/cassandra/src/java/org/apache/cassandra/service/StorageProxyMBean.java
/data/cassandra/src/java/org/apache/cassandra/service/ClientWarn.java
/data/cassandra/src/java/org/apache/cassandra/service/WriteResponseHandler.java
/data/cassandra/src/java/org/apache/cassandra/service/StorageService.java
/data/cassandra/src/java/org/apache/cassandra/service/NativeAccessMBean.java
/data/cassandra/src/java/org/apache/cassandra/service/RangeRelocator.java
/data/cassandra/src/java/org/apache/cassandra/service/EmbeddedCassandraService.java
/data/cassandra/src/java/org/apache/cassandra/service/PendingRangeCalculatorServiceEvent.java
/data/cassandra/src/java/org/apache/cassandra/service/PendingRangeCalcul

/data/cassandra/src/java/org/apache/cassandra/schema/ReplicationParams.java
/data/cassandra/src/java/org/apache/cassandra/schema/IndexMetadata.java
/data/cassandra/src/java/org/apache/cassandra/schema/Difference.java
/data/cassandra/src/java/org/apache/cassandra/schema/SchemaVersionVerbHandler.java
/data/cassandra/src/java/org/apache/cassandra/schema/CompactionParams.java
/data/cassandra/src/java/org/apache/cassandra/schema/SchemaAnnouncementDiagnostics.java
/data/cassandra/src/java/org/apache/cassandra/schema/Indexes.java
/data/cassandra/src/java/org/apache/cassandra/schema/ColumnMetadata.java


In [4]:
content = ""
for name in filenames:
    with open(name) as f:
        content += f.read() + "\n"

In [5]:
len(content)

13254762

In [6]:
content = content[:200000]

In [7]:
chars = sorted(list(set(content)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

print(chars)

total chars: 93
['\n', ' ', '!', '"', '#', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}']


In [8]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 120
step = 3
sentences = []
next_chars = []
for i in range(0, len(content) - maxlen, step):
    sentences.append(content[i: i + maxlen])
    next_chars.append(content[i + maxlen])
    
print('nb sequences:', len(sentences))

nb sequences: 66627


In [9]:
print('Vectorizing ...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
    
print('Vectorization is complete')

Vectorizing ...
Vectorization is complete


In [10]:
x.shape

(66627, 120, 93)

In [11]:
x.dtype

dtype('bool')

In [12]:
sys.getsizeof(x) / 10 ** 9

0.743557448

In [13]:
from tensorflow import keras

print('Build model...')

model = keras.Sequential()
model.add(keras.layers.InputLayer(input_shape=(maxlen, len(chars))))
model.add(keras.layers.LSTM(128))
model.add(keras.layers.Dense(len(chars), activation = "softmax"))

optimizer = keras.optimizers.Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer = optimizer)
model.summary()

W0929 17:08:24.524797 139938806101824 deprecation.py:506] From /home/abasar/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Build model...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               113664    
_________________________________________________________________
dense (Dense)                (None, 93)                11997     
Total params: 125,661
Trainable params: 125,661
Non-trainable params: 0
_________________________________________________________________


In [14]:
def sample(preds, temperature=1.0):
    """
    helper function to sample an index from a probability array
    """ 
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(content) - maxlen - 1)
    
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)
        generated = ''
        sentence = content[start_index: start_index + maxlen]
        print('----- Generating with seed: "' + sentence + '"')
        generated += sentence
        print(generated)
        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            generated += next_char
            sentence = sentence[1:] + next_char
        print(generated)

In [15]:
from keras.callbacks import LambdaCallback
import tensorflow as tf
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

with tf.device('/gpu:0'):
    model.fit(x, y, batch_size=128, epochs=250, callbacks=[print_callback])

Using TensorFlow backend.
W0929 17:08:24.963048 139938806101824 deprecation.py:323] From /home/abasar/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/250
66560/66627 [============================>.] - ETA: 0s - loss: 2.8997
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "er.info("adding range: ({},{}]", startToken, endToken);
                        ranges.add(new Range<>(startToken, endTo"
er.info("adding range: ({},{}]", startToken, endToken);
                        ranges.add(new Range<>(startToken, endTo
er.info("adding range: ({},{}]", startToken, endToken);
                        ranges.add(new Range<>(startToken, endToretentategestadatecenstatest ores                                                                                                                                                                                                                                                                                                                                                                               
----- diversity: 0.5
----- Generating with seed: "er.info("adding r

shot of a multiple column family from different keyspaces. A snapshot name must be specified.
     *
     * @param tag
               ale .kengepore. Sturt > In<)
                    reyseraw She.ged =Strageg>orep(en.ge.FeInelokens, mytoinGG)
                e caulis raplace %athr corntrresg th ixcSvoin());
              MessicpoineM.porasC();
;
      ilteraberampalimeHTWkass);
             orrfiom G] sinatetrblimm.gens()
           var.vetatdboecs requale.Endpoid<()ting ()
 {      * 
           y Engcutatats()
;
 
----- diversity: 1.2
----- Generating with seed: "shot of a multiple column family from different keyspaces. A snapshot name must be specified.
     *
     * @param tag
 "
shot of a multiple column family from different keyspaces. A snapshot name must be specified.
     *
     * @param tag
 
shot of a multiple column family from different keyspaces. A snapshot name must be specified.
     *
     * @param tag
   m HotstRenRewd?soyscabg e&lfrored"ctPing :kteg.PLEt6s"fic(eT<AdS

Epoch 6/250
66560/66627 [============================>.] - ETA: 0s - loss: 1.5683
----- Generating text after Epoch: 5
----- diversity: 0.2
----- Generating with seed: "();
        for (InetAddressAndPort ep : Gossiper.instance.getLiveMembers())
        {
            if (excludeDeadStates"
();
        for (InetAddressAndPort ep : Gossiper.instance.getLiveMembers())
        {
            if (excludeDeadStates
();
        for (InetAddressAndPort ep : Gossiper.instance.getLiveMembers())
        {
            if (excludeDeadStates());
                                                                                                                                                                                                                                                                                                                                                                                                           
----- diversity: 0.5
----- Generating with seed: "();
        for (

= keyspace.getColumnFamilyStore(tableName);
                // As there can be multiple column family from same keyspace in fort hoven")
            s to tfals ato aethatind
                                     n getreandre.tokenMbale m*));
                        a shewMBaty(emeve = nable remptit in the lagg_".Is geh)
    {
    }
    Vrearion poetTabnseringh wamrisg te hined in "cot resutrem toupsc
                                                              Beblacat.
   appictonfownRanges(getList<Streps, now Log
----- diversity: 1.2
----- Generating with seed: "= keyspace.getColumnFamilyStore(tableName);
                // As there can be multiple column family from same keyspace"
= keyspace.getColumnFamilyStore(tableName);
                // As there can be multiple column family from same keyspace
= keyspace.getColumnFamilyStore(tableName);
                // As there can be multiple column family from same keyspace +  e thes)
    *   nopaching stare. IONA_FDT)); 
 *
 {
          

...

apshot(String tag, Map<String, String> options, String... entities) throws IOException
    {
        boolean skipFlush = BDatState.getStute(value);
        }
        catch (INetG
                        "table-> scribury.RayCangisToken(keyspace))
            {
                alse remorst bull s& probutisilCException("Notes)
            logger.traceNator(LcallegeTcksForToSknowRebuars() dalle.value)
    {
        Ifendrese> lickeService.
iss ond hash."ASC_VELACINEADRibionicfifainsTrack)
    {
        alse
          
----- diversity: 1.2
----- Generating with seed: "apshot(String tag, Map<String, String> options, String... entities) throws IOException
    {
        boolean skipFlush ="
apshot(String tag, Map<String, String> options, String... entities) throws IOException
    {
        boolean skipFlush =
apshot(String tag, Map<String, String> options, String... entities) throws IOException
    {
        boolean skipFlush = BUASSTTOTOR.);
            break;
            if (keyspace == nul

Epoch 243/250
66560/66627 [============================>.] - ETA: 0s - loss: 0.0998
----- Generating text after Epoch: 242
----- diversity: 0.2
----- Generating with seed: "

            switch (moveName)
            {
                case VersionedValue.STATUS_BOOTSTRAPPING_REPLACE:
        "


            switch (moveName)
            {
                case VersionedValue.STATUS_BOOTSTRAPPING_REPLACE:
        


            switch (moveName)
            {
                case VersionedValue.STATUS_BOOTSTRAPPING_REPLACE:
                                                                                                                                                                                                                                                                                                                                                                                                                        
----- diversity: 0.5
----- Generating with seed: "

           

Map.Entry<InetAddressAndPort, UUID> entry : getTokenMetadata().getEndpointToHostIdMapForReading().entrySet())
            mapOut.pertion. deradionentanes.ringinging);
            preationManager.AllSSTableOpStatus Stoth(Ranges, cfimate.BIAN_NADRISExS.pex. valueFactory.nullanSeaviCyDescrimptraing joineTcUss, move.Seftencecector.
*

* Lives ftruarese,

            return Keyspace.open(tokens.add(endpoint);
        }

            return writat.
                                                                  hblecans
----- diversity: 1.2
----- Generating with seed: "Map.Entry<InetAddressAndPort, UUID> entry : getTokenMetadata().getEndpointToHostIdMapForReading().entrySet())
          "
Map.Entry<InetAddressAndPort, UUID> entry : getTokenMetadata().getEndpointToHostIdMapForReading().entrySet())
          
Map.Entry<InetAddressAndPort, UUID> entry : getTokenMetadata().getEndpointToHostIdMapForReading().entrySet())
            cfs + to fereningrano ing for the replication of a tit suemater 

Epoch 248/250
66560/66627 [============================>.] - ETA: 0s - loss: 0.0938
----- Generating text after Epoch: 247
----- diversity: 0.2
----- Generating with seed: "   public String getCommitLogLocation()
    {
        return FileUtils.getCanonicalPath(DatabaseDescriptor.getCommitLogL"
   public String getCommitLogLocation()
    {
        return FileUtils.getCanonicalPath(DatabaseDescriptor.getCommitLogL
   public String getCommitLogLocation()
    {
        return FileUtils.getCanonicalPath(DatabaseDescriptor.getCommitLogList(endpoint))
        {
            logger.debug("Resupt beeres to resulit tis node
        Map<String, Token = newRanges().rundTime() -> > getHosdddd());
            catch (Throws hatl. Node {} for bet, nod");

        // on check token token for a mighifitally we node the node whith hack a letecame this the names one the node if romdretion
        // we't chece token of now endpoint is.
       
----- diversity: 0.5
----- Generating with seed: "   public Str

            DatabaseDescriptor.getEndpointSnitch(),
                                                       streamStateStore, bstoness = new Range<>(mokByAtADDLLiveNtNativedTokens.vile)))
                logger.warn("Error pervilsed ()
            String[] strin(Token, null);
        }
    }

    /**
     * Endpoints filibly = SystemKeyspace.getSetesEndpoint(endpoint);
        return remot = net List<Keyspaces();

    /**
     * Endpoints is boct alreat state, end betien may the efing is conster.the the furret bey t
----- diversity: 1.2
----- Generating with seed: "            DatabaseDescriptor.getEndpointSnitch(),
                                                       streamStateSt"
            DatabaseDescriptor.getEndpointSnitch(),
                                                       streamStateSt
            DatabaseDescriptor.getEndpointSnitch(),
                                                       streamStateStore, Replicas, StrinagienRangestoress(), "S"TOS.g.LETORE.gR.getRNa

In [16]:
model.save("source-code-generator.h5")

In [18]:
open("source-code-generator-chars.txt", "w").write("".join(chars))

93